In [ ]:
pip install torch

In [1]:
import pandas as pd 
import numpy as np 
import torch
from torch import nn
from datetime import datetime
import itertools

ModuleNotFoundError: No module named 'torch'

mode = "train" | "predict"

In [ ]:
learning_rate = 1e-6
num_iterations = 2000
mode = "train"
save_model = False
load_model = False
model_save_path = "./model" #it will have automatically appended the current date
model_load_path = "model2024-01-25_16_38.pt"

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
device

'cuda'

In [5]:
# this is the initialization line in the original code:
# models[m] = getattr(model_list, args.models[m].model)()
class Classifier(nn.Module):
    def __init__(self, dim_input, num_classes):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(dim_input, 1500),
            nn.ReLU(),
            nn.Linear(1500, 780),
            nn.ReLU(),
            nn.Linear(780, num_classes),
        )
        self.softmax = nn.Softmax()

    # should return logits and features
    def forward(self, x):
        logits = self.model(x)
        prob = self.softmax(logits)
        return logits, prob

In [6]:
df = pd.read_pickle("./comet_samsum_train_pt1_z_entire.pkl")

In [7]:
ohe_dict= \
{'xNeed': np.array([1,0,0,0,0], dtype=np.float32),
 'xIntent': np.array([0,1,0,0,0], dtype=np.float32),
 'HinderedBy': np.array([0,0,1,0,0], dtype=np.float32),
 'xWant':  np.array([0,0,0,1,0], dtype=np.float32),
 'xReason':  np.array([0,0,0,0,1], dtype=np.float32)}

In [8]:
df["cs_type_ohe"] = df["cs_type"].map(ohe_dict)
preprocessed = df[["sample_id","sentence_id", "cs_type_ohe","cs_encoded", "cos_similary_cs_summmary"]]


In [9]:
set(preprocessed["cs_encoded"].transform(len))

{384}

In [10]:
preprocessed["complete_target_X"] =preprocessed[["cs_type_ohe","cs_encoded"]].apply(lambda row: np.append(row[0],row[1]), axis=1)

C:\Users\NonAv\AppData\Local\Temp\ipykernel_6220\3403830363.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed["complete_target_X"] =preprocessed[["cs_type_ohe","cs_encoded"]].apply(lambda row: np.append(row[0],row[1]), axis=1)


In [11]:
new_df = preprocessed.groupby(['sample_id','sentence_id']).agg(X=("complete_target_X",lambda x: list(itertools.chain.from_iterable(x))),Y=('cos_similary_cs_summmary',list))

new_df.reset_index() 

,sample_id,sentence_id,X,Y
0,13594093-1,0,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.04179999, -0.06145...","[0.4460858702659607, 0.10253575444221497, 0.39..."
1,13594093-1,1,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.07628163, 0.022710...","[0.46707096695899963, 0.625717043876648, 0.420..."
2,13594093-1,2,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.048776504, -0.0190...","[0.4569098949432373, 0.5190892815589905, 0.514..."
3,13594093-1,3,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.051447965, 0.00174...","[0.5159580707550049, 0.5018062591552734, 0.538..."
4,13594093-1,4,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.021506669, 0.00280...","[0.524988055229187, 0.4604058563709259, 0.5279..."
...,...,...,...,...
16994,13680254,12,"[0.0, 0.0, 1.0, 0.0, 0.0, -0.05868202, -0.0289...","[0.4484512209892273, 0.47776907682418823, 0.45..."
16995,13680255,0,"[0.0, 0.0, 1.0, 0.0, 0.0, -0.04349251, -0.0292...","[0.18616735935211182, 0.20535486936569214, 0.2..."
16996,13680255,1,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.006769986, 0.02239...","[0.6260838508605957, 0.5228524208068848, 0.626..."
16997,13680255,2,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.064037874, 0.04827...","[0.22459957003593445, 0.2936783730983734, 0.25..."


In [12]:
model = Classifier(dim_input=9725, num_classes=25).to(device)
if load_model:
    model.load_state_dict(torch.load(model_load_path))

In [13]:
sm =  nn.Softmax(dim=1)

In [14]:
X = torch.tensor(new_df["X"]).to(device)
Y = sm(torch.tensor(new_df["Y"])).to(device)

In [16]:
if mode == "train":
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    for t in range(num_iterations):
        # Forward pass: compute predicted y using operations on Tensors.
        optimizer.zero_grad()
        y_pred = model(X)

        # Compute and print loss using operations on Tensors.
        # Now loss is a Tensor of shape (1,)
        # loss.item() gets the scalar value held in the loss.
        loss = (y_pred - Y).pow(2).sum()
        if t % 100 == 99:
            print(t, loss.item())

        # Use autograd to compute the backward pass. This call will compute the
        # gradient of loss with respect to all Tensors with requires_grad=True.
        # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
        # the gradient of the loss with respect to a, b, c, d respectively.
        loss.backward()
        optimizer.step()

99 5.749238014221191
199 4.902899742126465
299 4.317054271697998
399 4.1678242683410645
499 4.067381381988525
599 3.984571933746338
699 3.9158260822296143
799 3.824004650115967
899 3.7488973140716553
999 3.9992377758026123
1099 3.659132480621338
1199 3.5584936141967773
1299 3.8796584606170654
1399 3.4996628761291504
1499 3.457339286804199
1599 3.503587245941162
1699 3.3169212341308594
1799 3.3714427947998047
1899 3.266228437423706
1999 3.2041015625


In [18]:
if save_model:
    torch.save(model.state_dict(), f"{model_save_path}{datetime.today().strftime('%Y-%m-%d_%H_%M')}.pt")

In [22]:
def test(X,Y, model, loss_fn):
    test_loss, correct = 0, 0
    with torch.no_grad():
        X, Y = X.to(device), Y.to(device)
        pred = model(X)
        test_loss = loss_fn(pred, Y)
        #correct = 
        for i,similarities in enumerate(pred):
            correct +=int(similarities.argmax() == Y[i].argmax())
    print(f"Test Error: \n Accuracy: {(100*correct/len(Y)):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [20]:
mode = "test"

In [23]:
if mode == "test" or mode == "train":
    test(X,Y, model, lambda y_pred, Y: (y_pred - Y).pow(2).sum())

Test Error: 
 Accuracy: 19.0%, Avg loss: 3.218035 

